## Prompt 1: Load and Describe Dataset

**Prompt:**  
Load the dataset by uploading an external CSV file and describe the dataset using
head(), tail(), info(), and describe().



In [ ]:
# Import required libraries
import pandas as pd
from google.colab import files
import io

# Upload CSV file
print("Please upload your CSV file.")
uploaded = files.upload()

# Read CSV into DataFrame
for file_name in uploaded.keys():
    print(f'User uploaded file "{file_name}"')
    df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))

print("Dataset loaded successfully into DataFrame 'df'.")


Please upload your CSV file.


In [ ]:
df.head()


In [ ]:
df.tail()

In [ ]:
df.info()


In [ ]:
df.describe()

PROMPT 2 Data Cleaning and Feature Engineering for Machine Learning

Prompt:

Perform data cleaning on the dataset to prepare it for machine learning.
The process should include:

Checking and quantifying missing values

Identifying and removing duplicate rows

Converting the timestamp column to a datetime object

Summarizing the cleaning steps and the current state of the dataset

Identify and quantify any missing values across all columns in the DataFrame to understand data completeness.

Reasoning:

To assess data quality, it is necessary to identify missing values that may impact model performance.
This is achieved by calculating the total number of null values for each column in the DataFrame df.

In [ ]:
# Check for missing values in each column
missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)


Subtask 2: Remove Duplicate Rows
Objective:

Identify and remove duplicate records to avoid biased or repeated data in machine learning models.

Reasoning:

Duplicate rows can distort model training by over-representing certain data points.
Removing duplicates ensures data integrity and improves model reliability.

In [ ]:
# Check number of duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows before removal: {duplicate_count}")

# Remove duplicate rows
df = df.drop_duplicates()

print("Duplicate rows removed successfully.")


Subtask 3: Convert Timestamp Column to Datetime
Objective:

Ensure the timestamp column is in datetime format for time-based analysis and feature extraction.

Reasoning:

Machine learning models and time-series analysis require timestamp data to be in a proper datetime format.
This allows extraction of features such as year, month, day, and hour.

In [ ]:
# Convert timestamp column to datetime format
# Convert order_date column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

print("date column converted to datetime format.")
display(df.head())
df.info()




Prompt 3: Use Machine Learning Models

Different machine learning algorithms capture different patterns in data.
Direct models learn independently, while hybrid models combine multiple
algorithms to improve prediction accuracy, robustness, and generalization.
Using both types allows comprehensive performance comparison.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score, mean_squared_error


In [ ]:
X = df.drop(columns=['financial_loss_usd', 'incident_id'])
y = df['financial_loss_usd']



In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Model 1: Linear Regression

Type: Regression

Purpose: Predicts continuous values (e.g., revenue)

Why used: Simple baseline model to understand linear relationships

In [ ]:
lr = Pipeline([
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)

print("Linear Regression R2:", r2_score(y_test, pred_lr))


Model 2: Ridge Regression

Type: Regularized Regression

Purpose: Handles multicollinearity using L2 regularization

Why used: Improves stability over Linear Regression

In [ ]:
ridge = Pipeline([
    ('preprocess', preprocessor),
    ('model', Ridge(alpha=1.0))
])

ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)

print("Ridge Regression R2:", r2_score(y_test, pred_ridge))


Model 3: Support Vector Regression (SVR)

Type: Kernel-based Model

Purpose: Captures complex non-linear patterns

Why used: Effective for high-dimensional data

In [ ]:
svr = Pipeline([
    ('preprocess', preprocessor),
    ('model', SVR(kernel='rbf'))
])

svr.fit(X_train, y_train)
pred_svr = svr.predict(X_test)

print("SVR R2:", r2_score(y_test, pred_svr))


Model 4: K-Nearest Neighbors (KNN) Regressor

Type: Instance-based Learning

Purpose: Predicts values based on nearest data points

Why used: Simple non-parametric approach for comparison

In [ ]:
knn = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5))
])

knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)

print("KNN R2:", r2_score(y_test, pred_knn))


Model 5: Voting Regressor (Hybrid Model 1)

Combination: Linear Regression + Decision Tree + Random Forest

Purpose: Aggregates predictions using averaging

Why used: Improves overall prediction stability

In [ ]:
voting = VotingRegressor(
    estimators=[
        ('lr', LinearRegression()),
        ('dt', DecisionTreeRegressor(random_state=42)),
        ('rf', RandomForestRegressor(n_estimators=50, random_state=42))
    ]
)

voting_model = Pipeline([
    ('preprocess', preprocessor),
    ('model', voting)
])

voting_model.fit(X_train, y_train)
pred_voting = voting_model.predict(X_test)

print("Voting Regressor R2:", r2_score(y_test, pred_voting))


Model 6: Boosting Model (Gradient Boosting / XGBoost-like)

Type: Sequential Hybrid Model

Purpose: Corrects previous model errors iteratively

Why used: High accuracy on structured sales data

In [ ]:
gboost = Pipeline([
    ('preprocess', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

gboost.fit(X_train, y_train)
pred_gb = gboost.predict(X_test)

print("Gradient Boosting R2:", r2_score(y_test, pred_gb))


In [ ]:
results = pd.DataFrame({
    "Model": [
        "Linear Regression",
        "Ridge",
        "SVR",
        "KNN",
        "Voting Regressor",
        "Gradient Boosting"
    ],
    "R2 Score": [
        r2_score(y_test, pred_lr),
        r2_score(y_test, pred_ridge),
        r2_score(y_test, pred_svr),
        r2_score(y_test, pred_knn),
        r2_score(y_test, pred_voting),
        r2_score(y_test, pred_gb)
    ]
}).round(3)

results




Prompt 4: Visualization of Model Performance

Prompt:
Create visual representations (bar graphs and line graphs) for the performance
of each machine learning model. Additionally, generate comparative charts
to analyze the performance of normal models versus hybrid models and
identify which category performs better for detecting patterns in the dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")



In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')

print(df[['date']].head())


In [ ]:
# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')

# Create month column
df['month'] = df['date'].dt.to_period('M')

# Count incidents per month
monthly_incidents = df.groupby('month').size()

# Plot
plt.figure(figsize=(10, 5))
monthly_incidents.plot(marker='o')

plt.title('Monthly Cybersecurity Incidents')
plt.xlabel('Month')
plt.ylabel('Number of Incidents')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(7, 5))

severity_counts = df['severity'].value_counts()

plt.bar(severity_counts.index, severity_counts.values)

plt.title('Incident Severity Distribution')
plt.xlabel('Severity Level')
plt.ylabel('Number of Incidents')
plt.tight_layout()
plt.show()


In [ ]:
top_attacks = df['attack_type'].value_counts().head(5)

plt.figure(figsize=(8, 5))
plt.bar(top_attacks.index, top_attacks.values)

plt.title('Top 5 Cyber Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


In [ ]:
avg_loss = df.groupby('severity')['financial_loss_usd'].mean()

plt.figure(figsize=(7, 5))
avg_loss.plot(kind='bar')

plt.title('Average Financial Loss by Severity')
plt.xlabel('Severity')
plt.ylabel('Average Loss (USD)')
plt.tight_layout()
plt.show()


SQL Queries


Load DataFrame into In-Memory SQLite
Load dataset into SQL database

Prompt:
Load the cybersecurity incidents DataFrame into an in-memory SQLite database so that SQL queries can be executed for analytical purposes.

Reasoning:
SQLite allows SQL-based analysis directly on pandas DataFrames without external database setup.

In [ ]:
import pandas as pd
import sqlite3

for col in df.columns:
    if isinstance(df[col].dtype, pd.PeriodDtype):
        df[col] = df[col].astype(str)

for col in df.columns:
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        df[col] = df[col].astype(str)

print("Column types cleaned")


In [ ]:
conn = sqlite3.connect(':memory:')

df.to_sql('incidents', conn, index=False, if_exists='replace')

print("Table 'incidents' created successfully")


In [ ]:
def run_sql(query):
    return pd.read_sql_query(query, conn)



SQL Query 1: Total Incident Count

Prompt:
Find the total number of incidents recorded in the dataset.

In [ ]:
def run_sql(query):
    return pd.read_sql_query(query, conn)


Query 1: Total Incidents

In [ ]:
query1 = """
SELECT COUNT(*) AS total_incidents
FROM incidents;
"""
run_sql(query1)


Query 2: Incidents by Severity

In [ ]:
query2 = """
SELECT severity, COUNT(*) AS incident_count
FROM incidents
GROUP BY severity
ORDER BY incident_count DESC;
"""
run_sql(query2)

Query 3: Incidents by Attack Type

In [ ]:
query3 = """
SELECT attack_type, COUNT(*) AS incident_count
FROM incidents
GROUP BY attack_type
ORDER BY incident_count DESC;
"""
run_sql(query3)


Query 4: Total Records Affected by Attack Type

In [ ]:
query4 = """
SELECT attack_type, SUM(records_affected) AS total_records
FROM incidents
GROUP BY attack_type
ORDER BY total_records DESC;
"""
run_sql(query4)


Query 5: Total Financial Loss by Industry


In [ ]:
query5 = """
SELECT industry, SUM(financial_loss_usd) AS total_loss
FROM incidents
GROUP BY industry
ORDER BY total_loss DESC;
"""
run_sql(query5)


Query 6: Top 5 Attack Detection Methods

In [ ]:
query6 = """
SELECT attack_detected_by, COUNT(*) AS count
FROM incidents
GROUP BY attack_detected_by
ORDER BY count DESC
LIMIT 5;
"""
run_sql(query6)


Query 7: Incidents Under Investigation vs Resolved vs Mitigated


In [ ]:
query7 = """
SELECT status, COUNT(*) AS count
FROM incidents
GROUP BY status;
"""
run_sql(query7)


Query 8: High Severity Incidents with Resolution Time > 150 hours

In [ ]:
query8 = """
SELECT incident_id, severity, resolution_time_hours, status
FROM incidents
WHERE severity IN ('High', 'Critical') AND resolution_time_hours > 150
ORDER BY resolution_time_hours DESC;
"""
run_sql(query8)


Query 9: Incidents with High Financial Loss > $4,000,000


In [ ]:
query10 = """
SELECT incident_id, industry, attack_type, severity, status
FROM incidents
WHERE severity = 'Low' AND status != 'Mitigated'
ORDER BY incident_id;
"""
run_sql(query10)



Query 10: Low Severity, Non-Mitigated Incidents


In [ ]:
query10 = """
SELECT incident_id, industry, attack_type, severity, status
FROM incidents
WHERE severity = 'Low' AND status != 'Mitigated'
ORDER BY incident_id;
"""
run_sql(query10)
